In [ ]:
import dask.dataframe as dd
import dask.bag as db
docs = dd.read_parquet("output.pq/", columns=['News_Id', 'News_Title', 'News_Text'])

In [ ]:
docs['News_Query'] = docs.apply(lambda row: [row['News_Title'], row['News_Text']], axis=1, meta=('News_Query', 'object'))

In [ ]:
import pyarrow as pa
schema = {
        "News_Id": pa.int32(),
        "News_Text": pa.string(),
        "News_Title": pa.string(),
        "News_Query": pa.list_(pa.string())
    }
docs.to_parquet("query.pq", schema = schema)

# Добавление запросов из LLM

In [ ]:
from transformers import pipeline

chatbot = pipeline("text-generation", model="microsoft/Phi-4-mini-instruct", max_new_tokens=256)

In [ ]:
import re

def parse_queries(llm_q: str) -> list[str]:
    # Разбиваем по строкам
    lines = llm_q.strip().split('\n')
    queries = []
    for line in lines:
        # Убираем номер и точку в начале строки, например "1. "
        query = re.sub(r'^\s*\d+\.\s*', '', line)
        queries.append(query.strip())
    return queries

promt_template = lambda news: f"""
        Вот текст новости:\n{news}"""

def generate_query(text: str) -> list[str]:
    messages = [
        {"role": "system", "content": """Тебе дают текст новости. 
         Сгенерируй 5 реалистичных поисковых запросов, которые могли бы ввести пользователи в поисковике, чтобы найти эту новость. 
         Учитывай, что пользователи часто не знают точных формулировок, могут использовать ключевые слова, имена, места, приблизительные описания и синонимы. 
         Запросы должны быть короткими (3-7 слов). Формулируй запросы разнообразно и естественно, как это делают реальные пользователи."""},
        {"role": "user", "content": promt_template(text)}
    ]
    llm_q = chatbot(messages)[0]['generated_text'][2]['content']
    return parse_queries(llm_q)

In [ ]:
sample_df = docs.sample(frac=0.0001, random_state=42)
# sample_df = docs.loc[0].compute()

In [ ]:
sample_df['News_Query'] = sample_df['News_Text'].map(generate_query,  meta=("News_Tokens", object)).compute()
print(sample_df.head())


In [ ]:
print(sample_df['News_Query'])

In [ ]:
import pyarrow as pa
schema = {
        "News_Id": pa.int64(),
        "News_Text": pa.string(),
        "News_Title": pa.string(),
        "News_Query": pa.string()
    }
sample_df.to_csv('llm-query.pq')